In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# %cd /content/drive/MyDrive/Video_Retrieval
# !git clone https://github.com/xuguohai/X-CLIP.git

from IPython.display import clear_output
%cd /content/drive/MyDrive/Video_Retrieval/X-CLIP
!pip install -r requirements.txt
clear_output()

## Downloads Dataset

### MSR-VTT

In [6]:
# from IPython.display import clear_output
# %cd /content/drive/MyDrive/Video_Retrieval/Dataset
# !wget https://github.com/ArrowLuo/CLIP4Clip/releases/download/v0.0/msrvtt_data.zip
# !wget https://www.robots.ox.ac.uk/~maxbain/frozen-in-time/data/MSRVTT.zip
# !unzip /content/drive/MyDrive/Video_Retrieval/Dataset/MSRVTT.zip
# !unzip /content/drive/MyDrive/Video_Retrieval/Dataset/msrvtt_data.zip
# clear_output()

## Download pretrain

In [ ]:
# %cd /content/drive/MyDrive/Video_Retrieval/X-CLIP
# # download CLIP（ViT-B/32） weight
# !wget -P ./modules https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt

# # download CLIP（ViT-B/16） weight
# !wget -P ./modules https://openaipublic.azureedge.net/clip/models/5806e77cd80f8b59890b7e101eabd078d9fb84e6937f9e85e4ecb61988df416f/ViT-B-16.pt

/content/drive/MyDrive/Video_Retrieval/X-CLIP
--2022-09-23 03:51:37--  https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.246.40, 13.107.213.40, 2620:1ec:bdf::40, ...
Connecting to openaipublic.azureedge.net (openaipublic.azureedge.net)|13.107.246.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353976522 (338M) [application/octet-stream]
Saving to: ‘./modules/ViT-B-32.pt’

ViT-B-32.pt         100%[===================>] 337.58M  40.0MB/s    in 7.7s    

2022-09-23 03:51:45 (43.7 MB/s) - ‘./modules/ViT-B-32.pt’ saved [353976522/353976522]

--2022-09-23 03:51:45--  https://openaipublic.azureedge.net/clip/models/5806e77cd80f8b59890b7e101eabd078d9fb84e6937f9e85e4ecb61988df416f/ViT-B-16.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.246.40, 13.107.213.40, 2620:1ec:bdf::40, ...
Connecti

## Training

In [ ]:
%cd /content/drive/MyDrive/Video_Retrieval/X-CLIP
# ViT-B/16
job_name="xclip_msrvtt_vit32"
# DATA_PATH="[Your MSRVTT data and videos path]"
!python -m torch.distributed.launch --nproc_per_node=1 \
    main_xclip.py --do_train --num_thread_reader=2 \
    --lr 1e-4 --batch_size=16  --batch_size_val 16 \
    --epochs=10  --n_display=1 \
    --train_csv /content/drive/MyDrive/Video_Retrieval/Dataset/msrvtt_data/MSRVTT_train.9k.csv \
    --val_csv /content/drive/MyDrive/Video_Retrieval/Dataset/msrvtt_data/MSRVTT_JSFUSION_test.csv \
    --data_path /content/drive/MyDrive/Video_Retrieval/Dataset/msrvtt_data/MSRVTT_data.json \
    --features_path /content/drive/MyDrive/Video_Retrieval/Dataset/MSRVTT/videos/all \
    --output_dir ckpts_dsw/${job_name} \
    --max_words 32 --max_frames 12 \
    --datatype msrvtt --expand_msrvtt_sentences  \
    --feature_framerate 1 --coef_lr 1e-3 \
    --freeze_layer_num 0  --slice_framepos 2 \
    --loose_type --linear_patch 2d --sim_header seqTransf \
    --pretrained_clip_name ViT-B/32 2>&1 | tee -a log/${job_name}

/content/drive/MyDrive/Video_Retrieval/X-CLIP
tee: log/: Is a directory
09/23/2022 05:54:22 - INFO -   Effective parameters:
09/23/2022 05:54:22 - INFO -     <<< batch_size: 16
09/23/2022 05:54:22 - INFO -     <<< batch_size_val: 16
09/23/2022 05:54:22 - INFO -     <<< cache_dir: 
09/23/2022 05:54:22 - INFO -     <<< coef_lr: 0.001
09/23/2022 05:54:22 - INFO -     <<< cross_model: cross-base
09/23/2022 05:54:22 - INFO -     <<< cross_num_hidden_layers: 4
09/23/2022 05:54:22 - INFO -     <<< data_path: /content/drive/MyDrive/Video_Retrieval/Dataset/msrvtt_data/MSRVTT_data.json
09/23/2022 05:54:22 - INFO -     <<< datatype: msrvtt
09/23/2022 05:54:22 - INFO -     <<< do_eval: False
09/23/2022 05:54:22 - INFO -     <<< do_lower_case: False
09/23/2022 05:54:22 - INFO -     <<< do_pretrain: False
09/23/2022 05:54:22 - INFO -     <<< do_train: True
09/23/2022 05:54:22 - INFO -     <<< epochs: 10
09/23/2022 05:54:22 - INFO -     <<< eval_frame_order: 0
09/23/2022 05:54:22 - INFO -     <<< exp

In [ ]:
%cd /content/drive/MyDrive/Video_Retrieval/X-CLIP
# ViT-B/16
job_name="xclip_msrvtt_vit16"
DATA_PATH="[Your MSRVTT data and videos path]"
!python -m torch.distributed.launch --nproc_per_node=1 \
    main_xclip.py --do_train --num_thread_reader=2 \
    --lr 1e-4 --batch_size=16  --batch_size_val 16 \
    --epochs=5  --n_display=10 \
    --train_csv /content/drive/MyDrive/Video_Retrieval/Dataset/msrvtt_data/MSRVTT_train.9k.csv \
    --val_csv /content/drive/MyDrive/Video_Retrieval/Dataset/msrvtt_data/MSRVTT_JSFUSION_test.csv \
    --data_path /content/drive/MyDrive/Video_Retrieval/Dataset/msrvtt_data/MSRVTT_data.json \
    --features_path /content/drive/MyDrive/Video_Retrieval/Dataset/MSRVTT/videos/all \
    --output_dir ckpts_dsw/${job_name} \
    --max_words 32 --max_frames 12 \
    --datatype msrvtt --expand_msrvtt_sentences  \
    --feature_framerate 1 --coef_lr 1e-3 \
    --freeze_layer_num 0  --slice_framepos 2 \
    --loose_type --linear_patch 2d --sim_header seqTransf \
    --pretrained_clip_name ViT-B/16 2>&1 | tee -a log/${job_name}